# CartPole Test   

In [1]:
!pip install ./pyansys_rl ./pyansys_gym

Processing ./pyansys_rl
Processing ./pyansys_gym
  Created wheel for pyansys-rl: filename=pyansys_rl-0.0.1-py3-none-any.whl size=7071 sha256=a9bd0f5c7a73eef1119ce50245e43dd6ac2fcd5784001965d402f51291953f00
  Stored in directory: /tmp/pip-ephem-wheel-cache-eneeb3gv/wheels/ab/f3/ff/bebff563baa460ac7ab74ef83360389d4ae6f5a05651e162f3
  Created wheel for pyansys-cartpole: filename=pyansys_cartpole-0.0.1-py3-none-any.whl size=10366 sha256=c4c078ac3120cf716c7ca521eb5e3d02e54f928a3f5b520de338cca7fd589076
  Stored in directory: /tmp/pip-ephem-wheel-cache-eneeb3gv/wheels/80/64/72/e71acde093f23c47c6e87f82f5841c0c889c0fcfdb32351491
Successfully built pyansys-rl pyansys-cartpole
  Attempting uninstall: pyansys-rl
    Found existing installation: pyansys-rl 0.0.1
    Uninstalling pyansys-rl-0.0.1:
      Successfully uninstalled pyansys-rl-0.0.1
  Attempting uninstall: pyansys-cartpole
    Found existing installation: pyansys-cartpole 0.0.1
    Uninstalling pyansys-cartpole-0.0.1:
      Successfully 

In [2]:
import os

import gym
import numpy as np

import pyansys_cartpole
from pyansys_dqn import dqn, dqn_runner, qn_keras
from pyansys_dqn.test_agents import RandomAgent, TrainedAgent

Using TensorFlow backend.
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python

In [3]:
np.set_printoptions(precision=4, suppress=True)

In [4]:
env_name = 'pyansys-CartPole-v0'
env = gym.make(env_name)

/home/jovyan/.local/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Random Agent
Here we create a simple test agent that behaves randomly and thus is not likely to succeed at the balancing task

In [5]:
agent = RandomAgent(env.action_space.n)

In [6]:
s = env.reset()
print(s)

[-0.0362  0.0226  2.1423  0.    ]


Below, notice how we inform the agent about each state transition with `agent.start_state(s)` or `agent.next_reading(s, r, done)` and then ask it to recommend an action with `agent.next_action()`.  We inform the environment this recommendation by feeding the method `env.step(a)`.  We do not expect these recommendations to be good because this agent selects at random from the choices 'left' and 'right', with equal probability.  A control algorithm that just flips a coin to select how to behave is usually not effective.  Thus, the pole should not stay balanced for long.

In [7]:
agent.start_state(s)
done, r_tot = False, 0
while not done:
    a = agent.next_action()
    s, r, done, _ = env.step(a)
    print('--->' if a else '<---', s)
    agent.next_reading(s, r, done, False)
    r_tot += r
print('total timesteps:', r_tot)

---> [-0.0351  0.1073  2.0279  0.0244]
---> [-0.031   0.3016  1.6733  0.0944]
<--- [-0.0252  0.2811  1.2656  0.0994]
<--- [-0.0215  0.0868  1.0034  0.0075]
---> [-0.0196  0.1064  0.7751  0.0149]
---> [-0.0155  0.3     0.4625  0.0934]
<--- [-0.0097  0.2807  0.1014  0.0768]
<--- [-0.006   0.0865 -0.1847  0.0211]
---> [-0.0041  0.1058 -0.3843  0.0033]
---> [-0.0001  0.3005 -0.689   0.0854]
<--- [ 0.0058  0.2811 -1.0856  0.0859]
<--- [ 0.0094  0.087  -1.3628  0.0028]
<--- [ 0.0092 -0.108  -1.35    0.0523]
---> [ 0.0073 -0.0879 -1.2011  0.0463]
<--- [ 0.0053 -0.1079 -1.0063  0.0273]
<--- [ 0.0012 -0.3025 -0.5993  0.0973]
<--- [-0.0068 -0.4969  0.0509  0.1906]
---> [-0.0165 -0.4779  0.8283  0.1678]
<--- [-0.0263 -0.4985  1.7263  0.1603]
---> [-0.0361 -0.478   2.5636  0.2085]
<--- [-0.0458 -0.4995  3.4062  0.1945]
<--- [-0.0578 -0.6956  4.5753  0.2757]
---> [-0.0715 -0.6752  5.7355  0.3699]
<--- [-0.0852 -0.6991  7.0174  0.3186]
<--- [-0.1012 -0.8945  8.6723  0.4648]
---> [-0.1189 -0.8749 10.

## Trained Agent
Now we create an agent that has been trained, i.e., that refers to a successful neural networks in order to decide how best to act. It is thus much more likely to perform well and balance the pole for a noticeably greater number of steps... all this despite having a random starting point for the system!

In [8]:
output_path = 'successful_runs/pyansys_cartpole'
output_name = 'pyansys_cartpole_00'
n_actions = 2
agent = TrainedAgent(output_path, output_name, env.action_space.n, env.observation_space.shape)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
s = env.reset()
print(s)

[ 0.01   -0.0067 -2.8594  0.    ]


Below, notice how we inform the agent about each state transition with `agent.start_state(s)` or `agent.next_reading(s, r, done)` and then ask it to recommend an action with `agent.next_action()`.  We follow its recommendation by feeding it into the environment in `env.step(a)`.  The recommendations should be pretty good because they stem from neural networks that store the information resulting from successful training and the pole should stay up longer, hopefully for the entirety of the episode (200 steps). 

In [10]:
agent.start_state(s)
done, r_tot = False, 0
while not done:
    a = agent.next_action()
    s, r, done, _ = env.step(a)
    print('--->' if a else '<---', s)
    agent.next_reading(s, r, done, False)
    r_tot += r
print('total timesteps:', r_tot)

<--- [ 0.009  -0.1072 -2.7467  0.0229]
<--- [ 0.0049 -0.3013 -2.3973  0.0896]
<--- [-0.0031 -0.4958 -1.7606  0.1498]
---> [-0.0128 -0.4758 -1.0122  0.1413]
---> [-0.0204 -0.2809 -0.4339  0.059 ]
---> [-0.024  -0.0859 -0.1443  0.0179]
---> [-0.0238  0.1098 -0.2224  0.0689]
<--- [-0.0218  0.0894 -0.4055  0.0949]
<--- [-0.022  -0.1049 -0.3964  0.0059]
---> [-0.0238 -0.0843 -0.3911  0.0178]
---> [-0.0236  0.1091 -0.4947  0.1114]
<--- [-0.0216  0.09   -0.608   0.0994]
<--- [-0.0217 -0.1036 -0.6867  0.01  ]
<--- [-0.0258 -0.2999 -0.4412  0.0337]
---> [-0.0316 -0.2795 -0.012   0.0514]
---> [-0.0352 -0.0843  0.1596  0.0117]
---> [-0.035   0.1096  0.0977  0.121 ]
<--- [-0.033   0.0908 -0.0703  0.0875]
<--- [-0.0331 -0.104  -0.1491  0.012 ]
---> [-0.035  -0.0845 -0.1138  0.0193]
---> [-0.0347  0.1104 -0.2228  0.0985]
<--- [-0.0327  0.0904 -0.4004  0.1129]
<--- [-0.0328 -0.1036 -0.4398  0.0128]
---> [-0.0347 -0.0837 -0.4568  0.0122]
---> [-0.0344  0.1102 -0.5646  0.125 ]
<--- [-0.0324  0.0911 -0.

## Epilogue
Try resuming a trained neural network of your own!